In [1]:
# Import statements and filename definitions
%matplotlib inline
from astropy import table
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
import pyspeckit

from full_spectrum_plot import plot_spectrum, plot_spectrum_line_ids
from spectral_ids import line_ids
results = '/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/'

In [ ]:
### PROOF OF CONCEPT
# Specify which spectral window you would like to look at
freq_spw = '97_spw27'
max_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.max.fits'
mean_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.mean.fits'
plot_spectrum(mean_fn)
mean_ids = line_ids(mean_fn)
plot_spectrum_line_ids(mean_fn, mean_ids)
plot_spectrum(max_fn)
max_ids = line_ids(max_fn)
plot_spectrum_line_ids(max_fn, max_ids)

In [ ]:
### ALL BAND 3 SPECTRAL WINDOWS
for freq_spw in ['87_spw25','87_spw102','89_spw27','89_spw104',
                 '91_spw25','91_spw102','93_spw27','93_spw104',
                 '95_spw25','97_spw27','98_spw29','98_spw106',
                 '99_spw25','99_spw31','99_spw108','101_spw27',
                 '102_spw23','102_spw29','102_spw106','103_spw31',
                 '103_spw108','104_spw25','106_spw29','107_spw31',
                 '110_spw29','111_spw31','112_spw27','114_spw29']:
    print('------------------------------------------------')
    print('-------- '+freq_spw)
    print('------------------------------------------------')
    max_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.max.fits'
    mean_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.mean.fits'
#     print('Mean spectrum:')
#     plot_spectrum(mean_fn)
    mean_ids = line_ids(mean_fn)
    print('Mean spectrum with IDs:')
    plot_spectrum_line_ids(mean_fn, mean_ids)
#     print('Max spectrum:')
#     plot_spectrum(max_fn)
    max_ids = line_ids(max_fn)
    print('Max spectrum with IDs:')
    plot_spectrum_line_ids(max_fn, max_ids)

In [ ]:
### ACES PREP
band_3 = ['87_spw25','87_spw102','89_spw27','89_spw104',
          '91_spw25','91_spw102','93_spw27','93_spw104',
          '95_spw25','97_spw27','98_spw29','98_spw106',
          '99_spw25','99_spw31','99_spw108','101_spw27',
          '102_spw23','102_spw29','102_spw106','103_spw31',
          '103_spw108','104_spw25','106_spw29','107_spw31',
          '110_spw29','111_spw31','112_spw27','114_spw29']
all_lines_max = []
all_lines_mean = []
for freq_spw in band_3:
    print('------------------------------------------------')
    print('---------------- '+freq_spw)
    print('------------------------------------------------')
    max_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.max.fits'
    mean_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.mean.fits'
    max_ids = line_ids(max_fn)
    all_lines_max.append(max_ids)
    mean_ids = line_ids(mean_fn)
    all_lines_mean.append(mean_ids)

In [ ]:
# Get stats about max spectra
all_lines_max_stack = table.vstack(all_lines_max)
all_lines_max_stack.sort('Freq')
max_spec_freq = all_lines_max_stack['Freq']
# Get stats about mean spectra
all_lines_mean_stack = table.vstack(all_lines_mean)
mean_spec_freq = all_lines_mean_stack['Freq']
# Need also number of lines above a threshold density. Maybe use EU_K?

In [ ]:
### Plots for max spectra
## Total number of lines in max spectra
bins = np.arange(np.floor(np.min(max_spec_freq)), np.ceil(np.max(max_spec_freq))+1, 1.)
counts, bins1 = np.histogram(max_spec_freq, bins=bins)
plt.step(bins[:-1], counts, where='post')
plt.xlabel("Frequency (GHz)")
plt.ylabel("Number of lines per 1 GHz bin")
plt.title("Total number of lines (max spectra)")
plt.show()

## Total intensity of lines in max spectra
# Do some quick binning
max_spec_tot_int_sums = []
for bin_number in np.arange(len(bins)-1):
    freq_lo, freq_hi = bins[bin_number], bins[bin_number+1] # Get edges of frequency bin
    lines_in_bin = all_lines_max_stack[(all_lines_max_stack['Freq']>=freq_lo) & 
                                       (all_lines_max_stack['Freq']<=freq_hi)]
    max_spec_tot_int_sums.append(np.sum(lines_in_bin['TotalIntensity']))
plt.step(bins[:-1], max_spec_tot_int_sums, where='post')
plt.xlabel("Frequency (GHz)")
plt.ylabel("Total intensity of lines (Jy/beam/bin)")
plt.title("Total intensity of lines (max spectra)")
plt.show()

## Average peak intensity of lines in max spectra
# Do some quick binning
max_spec_peak_int_avgs = []
for bin_number in np.arange(len(bins)-1):
    freq_lo, freq_hi = bins[bin_number], bins[bin_number+1] # Get edges of frequency bin
    lines_in_bin = all_lines_max_stack[(all_lines_max_stack['Freq']>=freq_lo) & 
                                       (all_lines_max_stack['Freq']<=freq_hi)]
    if len(lines_in_bin) != 0:
        max_spec_peak_int_avgs.append(np.mean(lines_in_bin['PeakIntensity']))
    else:
        max_spec_peak_int_avgs.append(0.)
plt.step(bins[:-1], max_spec_peak_int_avgs, where='post')
plt.xlabel("Frequency (GHz)")
plt.ylabel("Average peak intensity of lines (Jy/beam/bin)")
plt.title("Average peak intensity of lines (max spectra)")
plt.show()

In [ ]:
### Plots for mean spectra
# Total number of lines in mean spectra
bins = np.arange(np.floor(np.min(mean_spec_freq)), np.ceil(np.max(mean_spec_freq))+1, 1.)
counts, bins1 = np.histogram(mean_spec_freq, bins=bins)
plt.step(bins[:-1], counts, where='post')
plt.xlabel("Frequency (GHz)")
plt.ylabel("Number of lines per 1 GHz bin")
plt.title("Total number of lines (mean spectra)")
plt.show()

## Total intensity of lines in mean spectra
# Do some quick binning
mean_spec_tot_int_sums = []
for bin_number in np.arange(len(bins)-1):
    freq_lo, freq_hi = bins[bin_number], bins[bin_number+1] # Get edges of frequency bin
    lines_in_bin = all_lines_mean_stack[(all_lines_mean_stack['Freq']>=freq_lo) & 
                                        (all_lines_mean_stack['Freq']<=freq_hi)]
    mean_spec_tot_int_sums.append(np.sum(lines_in_bin['TotalIntensity']))
plt.step(bins[:-1], mean_spec_tot_int_sums, where='post')
plt.xlabel("Frequency (GHz)")
plt.ylabel("Total intensity of lines (Jy/beam/bin)")
plt.title("Total intensity of lines (mean spectra)")
plt.show()

## Average peak intensity of lines in max spectra
# Do some quick binning
mean_spec_peak_int_avgs = []
for bin_number in np.arange(len(bins)-1):
    freq_lo, freq_hi = bins[bin_number], bins[bin_number+1] # Get edges of frequency bin
    lines_in_bin = all_lines_mean_stack[(all_lines_mean_stack['Freq']>=freq_lo) & 
                                        (all_lines_mean_stack['Freq']<=freq_hi)]
    if len(lines_in_bin) != 0:
        mean_spec_peak_int_avgs.append(np.mean(lines_in_bin['PeakIntensity']))
    else:
        mean_spec_peak_int_avgs.append(0.)
plt.step(bins[:-1], mean_spec_peak_int_avgs, where='post')
plt.xlabel("Frequency (GHz)")
plt.ylabel("Average peak intensity of lines (Jy/beam/bin)")
plt.title("Average peak intensity of lines (mean spectra)")
plt.show()

In [ ]:
### Focus on spectral windows with greatest line density
for freq_spw in ['87_spw25','102_spw106']:
    max_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.max.fits'
    mean_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.mean.fits'
    print("------------------------ Max spectrum ------------------------")
    max_ids = line_ids(max_fn)
    plot_spectrum_line_ids(max_fn, max_ids)
    print("------------------------ Mean spectrum ------------------------")
    mean_ids = line_ids(mean_fn)
    plot_spectrum_line_ids(mean_fn, mean_ids)

In [ ]:
freq_spw = '87_spw25'
mean_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.mean.fits'
line_ids(mean_fn)

In [ ]:
freq_spw = '102_spw106'
mean_fn = results+'spectra/source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.mean.fits'
line_ids(mean_fn)